# Echantillonnage compressif

In [ ]:
import numpy as np
import math
import scipy.fftpack as fft
import matplotlib.pyplot as plt
import numpy.linalg as npl

Récemment (début années 2004-présent), de nouveaux concepts et théorèmes ont été développés et risquent de 
révolutionner à relativement court terme la fabrication de certains appareils de mesure numériques (microphones, imageurs, analyseurs de spectres,...). 
Ces nouvelles techniques sont couramment appelées échantillonnage compressif, "compressive sampling" ou encore "compressed sensing". 

## 1. Le théorème de Shannon

Aujourd'hui, presque tous les appareils de mesure reposent sur le théorème de Shannon. Celui-ci (vous l'avez déjà vu en 2ème année) peut s'énoncer ainsi : 
> Soit $g:\mathbb{R}\to \mathbb{R}$ une fonction de $L^2(\mathbb{R})$. Si sa transformée de Fourier $\hat g$ a un support contenu dans l'intervalle $[-f_M, f_M]$, alors en l'échantillonnant à une fréquence d'échantillonnage $f_e\geq 2f_M$, on peut la reconstruire exactement.

Les instruments de mesures qui reposent sur ce théorème sont donc construits suivant le principe : 
>Filtre passe-bas $\rightarrow$ Echantillonnage à une fréquence $f>2f_M$ $\rightarrow$ Interpolation sinc

Pour beaucoup d'applications, ce principe présente deux défauts majeurs :
* Les signaux sont rarement naturellement à spectre borné, et on perd donc l'information haute-fréquence en effectuant un filtrage passe-bas.
* Pour beaucoup de signaux, il faut choisir une très haute fréquence d'échantillonnage pour obtenir un résultat satisfaisant. 
Ceci implique que les données à stocker ont une taille très importante et qu'il faut les compresser après coup (par exemple : jpeg).




## 2. L'échantillonnage compressif

**1. Principe général**

L'idée sous jacente à l'échantillonnage compressif est de réaliser la compression dès l'acquisition.
Supposons que le signal $x\in \mathbb{R}^n$ que l'on souhaite mesurer s'écrive comme une combinaison linéaire de la forme :
\begin{equation}
(1)~~~~~~~~~~~ x=\sum_{i=1}^m\alpha_i \psi_i
\end{equation}
où $\psi_i\in \mathbb{R}^n, \ i=1..m$, sont des "fonctions de base" (en traitement d'images, ces fonctions pourraient être des ondelettes, en traitement du son, des ondelettes ou des atomes de Fourier, pour certaines applications, on pourrait imaginer des splines...} et $\alpha_i\in \mathbb{R}$ sont des coefficients. 
On peut réécrire l'équation (1) sous la forme matricielle condensée :
$$
x=\Psi \alpha \ \ \textrm{où } \ \ \alpha=\begin{pmatrix} \alpha_1 \\ \vdots \\ \alpha_m \end{pmatrix}\ \ \textrm{et} \ \ \Psi=\begin{pmatrix} \psi_1,\psi_2,..., \psi_m\end{pmatrix}.
$$
Pour pouvoir reconstruire tous les éléments de $\mathbb{R}^n$, on suppose généralement que la matrice $\Psi$ est une matrice surjective (ainsi, la famille  des $(\Psi_i)_i$ est génératrice), ce qui implique que $m\geq n$. Dans le langage du traitement d'image, on dit alors que $\Psi$ est un frame (une base si $m=n$).

L'échantillonnage compressif repose sur l'hypothèse suivante : les signaux $x$ que l'on souhaite mesurer sont parcimonieux, 
c'est-à-dire que la majorité des coefficients $\alpha_i$ dans (1) sont nuls ou encore que 
$$\#\{\alpha_i\neq 0, i=1..m\}\ll n.$$
On va voir que cette hypothèse permet - dans certains cas - de réduire drastiquement le nombre de mesures par rapport au théorème de Shannon avec en contre-partie, le besoin de résoudre un problème d'optimisation pour reconstruire la donnée. L'objectif de ce travail est de résoudre le problème d'optimisation résultant.

Le principe de l'acquisition du signal $x$ est le suivant :

- On effectue un petit nombre $p\ll n$ de mesures linéaires du signal $x$ inconnu. On note ces mesures $y_i$, et comme elles sont linéaires par rapport à $x$, il existe pour chaque $i$ un vecteur $a_i\in \mathbb{R}^n$ tel que 
$$y_i=\langle a_i, x\rangle, i=1..p.$$ On peut aussi écrire cette opération de mesure sous la forme condensée :
$$
y=Ax\ \ \textrm{où } \ \ y=\begin{pmatrix} y_1 \\ \vdots \\ y_p\end{pmatrix} \ \ \textrm{et} \ \ A=\begin{pmatrix} a_1^T\\a_2^T\\ \vdots
\\ a_p^T\end{pmatrix}.
$$
- On reconstruit le signal $x$ en résolvant le problème contraint suivant :

$$
(2)~~~~~~~~~~~ \mbox{Trouver } \alpha^\star \mbox{ solution de: }\displaystyle\min_{\alpha \in \mathbb{R}^m, A\Psi\alpha=y} \|\alpha\|_0
$$

où $\|\cdot\|_0$ est la norme de comptage, aussi appelée norme $l^0$ définie par : 
$$
\|\alpha\|_0=\#\{\alpha_i\neq 0, i=1..m\}.
$$
Autrement dit, l'idée est la suivante : on  cherche $\alpha^\star$, le signal le plus parcimonieux dans le frame $\Psi$, parmi les signaux qui peuvent donner lieu aux mesures $y$. 
Après avoir trouvé $\alpha^\star$, on recouvre $\tilde x$, une approximation du signal $x$ en calculant $\tilde x=\Psi\alpha^\star$.



**2. Simplification du problème d'optimisation**

Le problème précédent est un problème combinatoire NP-complet, ce qui signifie que trouver $\alpha$ peut demander un temps exponentiel en fonction de $n$, la dimension du signal. Pour le résoudre en pratique, il est souvent remplacé par : 

$$
(3)~~~~~~~~~~~  \mbox{Trouver } \alpha^*\in \displaystyle\arg\min_{\alpha \in \mathbb{R}^m, A\Psi\alpha=y} \|\alpha\|_1
$$

où $\|\alpha\|_1=\sum_{i=1}^m|\alpha_i|$ est la norme $l^1$ de $\alpha$. On peut dans certains cas montrer que les solutions de (2) et de (3) sont identiques. 

Un appareil de mesure n'étant jamais parfait, il est impossible de mesurer exactement $y_i=\langle a_i, x\rangle$. 
Le vecteur $y$ est bruité et la contrainte $A\Psi\alpha=y$ est trop forte. Elle est donc généralement relaxée et le problème devient : 

>$$(4)~~~~~~~~~~~  \mbox{Trouver } \alpha^*\in \arg\min_{\alpha \in \mathbb{R}^m} \|\alpha\|_1 + \frac{\sigma}{2} \|A\Psi\alpha-y\|_2^2.$$

Si $\sigma$ tend vers $0$, la solution du problème (4) tend vers une solution du problème (3). C'est le problème (4) que nous allons résoudre dans ce travail. Dans la suite , on notera $F$ la fonction :
$$
F(\alpha)=\|\alpha\|_1 + \frac{\sigma}{2} \|A\Psi\alpha-y\|_2^2.
$$

Pour conclure cette introduction à l'échantillonnage compressif, notons que de façon similaire au théorème de Shannon, on dispose d'une condition de reconstruction exacte :

> Supposons que :
* $x=\displaystyle\sum_{i=1}^m\alpha_i\psi_i\in \mathbb{R}^n$ avec $\|\alpha\|_0=k$.
* On effectue $p$ mesures linéaires de $x$ avec $p\geq C \cdot k \cdot \log(n)$, où $C=20$.
* On choisit les coefficients de la matrice $A\in \mathcal{M}_{p,n}$ de façon **aléatoire** (e.g. on peut choisir les coefficients $a_{i,j}$ de $A$ de façon indépendante suivant une loi normale.)

> Alors, la résolution du problème (3) permet de reconstruire $x$ **exactement** avec une très grande probabilité 

L'expérience a montré qu'en pratique, il suffit en général de $p=2k$ mesures pour reconstruire le signal exactement en grande dimension !

3. Préliminaires théoriques
----------------------------

Commençons par remarquer que les problèmes (3) et (4) sont convexes (contraintes convexes et fonctions convexes) tandis que le problème (2) ne l'est pas. En revanche, aucun des trois problèmes n'est différentiable.

**Q1.** Soit $J(\alpha) =\frac{\sigma}{2}\|A\Psi \alpha - y\|_2^2$. Calculez $\nabla J(\alpha)$.

### Réponse: 

$\nabla J(\alpha)$ = $(A\Psi)^T\sigma(A\Psi \alpha - y)$. 


**Q2.** Montrer que la fonction $J$ est de classe $C^1$ à gradient Lipschitz et calculer un majorant $L$ de la constante de Lipschitz de $\nabla J$ en fonction de $|||A|||$, de $|||\Psi |||$ et de $\sigma$.


 ### Réponse: 


Il s'agit d'une composition entre une application linéaire qu'on peut noter L,tel que L: x -> $A\Psi x -y$ et une norme $||$ qui est  $C^\infty$ donc J est $C^\infty$.

$\|\nabla J(\alpha_1)-\nabla J(\alpha_2)\| \leq \sigma|||(A\Psi)^T||| \|(A\Psi (\alpha_1 - \alpha_2)\| \leq 2\sigma |||A|||^2\|\alpha_1-\alpha_2\|$. Donc $L=2\sigma |||A|||^2$.

**Construction de l'algorithme**

On note $\alpha^k$ l'itéré courant. En appliquant le lemme de Nesterov à la fonction $J$, on a:
$$\forall \alpha\in \mathbb{R}^m,~J(\alpha)\leq {J(\alpha^k) + \langle \nabla J(\alpha^k), \alpha-\alpha^k\rangle + \frac{L}{2}\|\alpha-\alpha^k\|_2^2}.
$$

En posant $\phi(\alpha,\alpha^k)=J(\alpha^k) + \langle \nabla J(\alpha^k), \alpha-\alpha^k\rangle + \frac{L}{2}\|\alpha-\alpha^k\|_2^2 +\|\alpha\|_1$, on a alors:

$$\forall \alpha\in \mathbb{R}^m,~F(\alpha) = J(\alpha)+\|\alpha\|_1\leq \phi(\alpha,\alpha^k),$$

avec : $\phi(\alpha^k,\alpha^k) = F(\alpha^k)$.

Cette inégalité motive alors l'algorithme de descente suivant:

$$\alpha^{k+1}=\displaystyle\arg\min_{\alpha\in \mathbb{R}^m} \phi(\alpha,\alpha^k).$$

**Q3.** Montrer que l'algorithme s'écrit de façon équivalente sous la forme:
$$\alpha^{k+1} = \mbox{prox}_{\frac{1}{L}\|\cdot\|_1}\left(\alpha^k - \frac{1}{L}\nabla J(\alpha^k)\right).$$



### Réponse: 

On a que
$$\alpha^{k+1}=\displaystyle\arg\min_{\alpha\in \mathbb{R}^m} \phi(\alpha,\alpha^k)=\displaystyle\arg\min_{\alpha\in \mathbb{R}^m}(J(\alpha^k) + \langle \nabla J(\alpha^k), \alpha-\alpha^k\rangle + \frac{L}{2}\|\alpha-\alpha^k\|_2^2 +\|\alpha\|_1)$$

D'autre part on a 
$$\begin{align}
\mbox{prox}_{\frac{1}{L}\|\cdot\|_1}\left(\alpha^k - \frac{1}{L}\nabla J(\alpha^k)\right)&=\displaystyle\arg\min_{\alpha\in \mathbb{R}^m}(\frac{1}{L}\|\alpha\|_1+\frac{1}{2}\|\alpha-\alpha^k+\frac{1}{L}\nabla J(\alpha^k)\|_2^2)\\
&=\displaystyle\arg\min_{\alpha\in \mathbb{R}^m}(\|\alpha\|_1+\frac{L}{2}\|
\alpha-\alpha^k\|_2^2+\frac{1}{2L}\|\nabla J(\alpha^k)\|_2^2+\langle\alpha-\alpha^k,\nabla J(\alpha^k)\rangle)
\end{align}$$

Comme on a  $\frac{1}{2L}\|\nabla J(\alpha^k)\|_2^2$ et $J(\alpha^k)$ sont indépendant de $\alpha$, on a bien que 


$$\alpha^{k+1}=\mbox{prox}_{\frac{1}{L}\|\cdot\|_1}\left(\alpha^k - \frac{1}{L}\nabla J(\alpha^k)\right)$$

**Q4.** En déduire la formule analytique donnant $\alpha^{k+1}$ en fonction de $\alpha^k$.
### Réponse: 

On en deduit que $$\begin{align}
\alpha^{k+1}&=\displaystyle\arg\min_{\alpha\in \mathbb{R}^m}(\frac{1}{L}\|\alpha\|_1+\frac{1}{2}\|\alpha-\alpha^k+\frac{1}{L}\nabla J(\alpha^k)\|_2^2)\\
&=\displaystyle\arg\min_{\alpha\in \mathbb{R}^m}(\frac{1}{L}\sum_{i=1}^{m}|\alpha_i|+\frac{1}{2}\sum_{i=1}^{m}(\alpha_i-\alpha_i^k+\frac{1}{L}\nabla J (\alpha_k)_i)^2)
\end{align}$$




**Q5.** De quelle quantité la fonction coût $F(\alpha)$ décroît-elle à chaque itération ?

### Réponse: 

$$F(\alpha^{k+1})-F(\alpha^k)=J(\alpha^{k+1})+\|\alpha^{k+1}\|_1-J(\alpha^{k})+\|\alpha^{k}\|_1\leq \phi(\alpha^{k+1},\alpha^k)-\phi(\alpha^k,\alpha^k)\leq 0$$

4. Partie expérimentale
------------------------

Dans ce travail, on va chercher à reconstruire un signal unidimensionnel $x~:~[0,1]~\rightarrow~\mathbb{R}$ de la forme :
$$
x(t)= \alpha_{k}\delta_{k/n}(t)+\sum_{k=1}^n\alpha_{k+n}\cos\left(\frac{2k\pi}{n} t\right)
$$ 
on a donc $m=2n$.

In [ ]:
## Initialisations
n=500            #Taille de l'echantillon
t=np.linspace(0,1,n) #On definit un signal sur [0,1]

## Generation du signal
x=np.zeros(n)
tmp=np.zeros(n)
#On ajoute deux cosinus
tmp[350]=4
x+=fft.idct(tmp,norm='ortho')  
tmp=np.zeros(n)
tmp[150]=-3  
x+=fft.idct(tmp,norm='ortho')
#On ajoute deux diracs
x[int(n/3)]=0.2;    #Tester 0.5
x[int(2*n/3)]=-0.3; #Tester -1
x0=np.copy(x)

plt.plot(t,x,'b')
plt.show()
## Mesure du signal
p=20*4       #Nombre de mesures
A=np.random.randn(p,n) #La matrice de mesure
y=A.dot(x)        #Les mesures

Le code *Generesignal.py* génère un signal discret $x$ qui peut être vu comme une combinaison linéaire de cosinus à différentes fréquences et de diracs. Ce signal n'est pas parcimonieux dans la base canonique des diracs (car il faut à peu près $n$ diracs pour représenter un cosinus) et il n'est pas parcimonieux dans la base des sinus (il faut faire une combinaison linéaire de $n$ cosinus pour représenter un dirac).

Par contre, ce signal est parcimonieux dans un frame qui est l'union de la base canonique et de la base des cosinus. 
Dans ce frame, il suffit en effet de $4$ coefficients non nuls pour reconstruire parfaitement le signal.

> On choisira donc le frame représenté par une matrice $\Psi=(I,C) \in \mathcal{ M}_{2n,n}(\mathbb{R})$ o\`u $C$ est une base de cosinus à différentes fréquences.

### 4.1. Implémentation de l'itération proximale

**Q6** Implémentez l'opérateur linéaire $\Psi$ et son adjoint $\Psi^*$. 

Pour $\Psi$, vous vous servirez de la fonction $dct$ de Python dans la libraire scipy.fftpack qui calcule la transformée en cosinus discret d'un vecteur. Vous ferez attention à préciser *norm='ortho'* dans les options de la $dct$ pour que $idct$ soit bien l'opération inverse de $dct$.

Pour $\Psi^*$, vous utiliserez le fait que la $dct$ est une isométrie quand on précise \textit{norm='ortho'} dans les options de $dct$.

In [ ]:
## Linear function Psi
## (combination of sines and diracs)
 
def Psi(alpha) :
    n=len(alpha)
    x=np.zeros(int(n/2))
    x=alpha[:int(n/2)]+fft.idct(alpha[int(n/2):], norm ='ortho')
    return x


## The transpose of Psi
def PsiT(x) : 
    alpha=np.concatenate((x,fft.dct(x,norm='ortho')),axis=0)
    return alpha

**Q7** Implémentez l'algorithme proximal dans la fonction *RestoreX* avec les notations suivantes:
* $A$ est la matrice d'échantillonnage.
* $y$ est le vecteur de mesures.
* $sigma$ est un paramètre du modèle.
* $nit$ est le nombre d'itérations.
* $alpha$ est la solution approximative du problème (4).
* $x$ est donné par Psi $(\alpha)$.
* $CF$ est la fonction coût à chaque itération de l'algorithme. 

In [ ]:
def gradJ(u, sigma, A, y) : 
    return sigma * PsiT( A.T.dot( A.dot( Psi(u) ) - y))

def cf(u, sigma, A, y) : 
    return (sigma/2) * npl.norm(A.dot(Psi(u)) - y )**2 + npl.norm(u,1)

In [ ]:
## Prox of the l1−norm 
def prox(alpha,gamma) :
    beta=np.sign(alpha)*np.maximum(np.abs(alpha)-gamma,0)
    return beta

def RestoreX(A,y,sigma,nit) :
    n=np.shape(A)[1]
    x=np.zeros(nit*n) 
    alpha=np.zeros(2*n)
    CF=np.zeros(nit)
    iter=0
    while iter<nit:
        L=2*sigma*npl.norm(A)**2
        alpha=prox(alpha-(1/L)*sigma*PsiT(A.T.dot(A.dot(Psi(alpha))-y)),1/L)
        x[n*iter:n*(iter+1)]=Psi(alpha)
        CF[iter]=npl.norm(alpha,1)+(sigma/2)*npl.norm(A.dot(Psi(alpha))-y)**2
        iter+=1
    return (alpha,x,CF)

def RestoreX_with_List_alpha(A,y,sigma,nit) :
    n=np.shape(A)[1]
    x=np.zeros(nit*n) 
    alpha=np.zeros(2*n)
    CF=np.zeros(nit)
    Al=[alpha]
    iter=0
    while iter<nit:
        L=2*sigma*npl.norm(A)**2
        alpha=prox(alpha-(1/L)*sigma*PsiT(A.T.dot(A.dot(Psi(alpha))-y)),1/L)
        Al+=[alpha]
        x[n*iter:n*(iter+1)]=Psi(alpha)
        CF[iter]=npl.norm(alpha,1)+(sigma/2)*npl.norm(A.dot(Psi(alpha))-y)**2
        iter+=1
    return (Al,x,CF)


**Q8.** Testez votre algorithme ! Les paramètres $sigma$ et $nit$ sont des à choisir par vous-même (il faut en pratique beaucoup d'itérations pour converger). Vous pourrez observer la façon dont la suite $\alpha^k$ se comporte au fur et à mesure des itérations.


Dans cette partie, nous fixons $\sigma$ à 0.05 et $nit$ à 5000 itérations.
Nous commençons d'abord par tester le comportement de la suite $\alpha^k$ au fur et à mesure des itérations. Pour ce faire, nous décidons de faire des plot de plusieurs itérations espacées de 500. Ce qui nous permet d'obtenir les 11 courbes ci-dessous. 

In [ ]:
sigma=0.05
nit=5000
(alpha,xtilde,CF)=RestoreX_with_List_alpha(A,y,sigma,nit)
fig, (AXE) = plt.subplots(2, 5)
fig.suptitle(r"$\sigma$ = "+ "{:.1e}".format(sigma) + " / nit =" + "{:}".format(nit),fontsize=40)
fig.set_size_inches(40, 15)
plt.subplots_adjust(bottom=0.1)
Old2New = lambda i : np.array([i//5 , i%5])
for i in range(10):
    axe = AXE[Old2New(i)[0],Old2New(i)[1]]
    axe.set_title(" iteration = "+"{:}".format(i+1))
    axe.plot(alpha[500*i])






Nous débutons avec le vecteur $\alpha$ qui contient que des 0.

Après chaque itération, le nombre de coefficient non nul augmente. 

On peut supposer que l'algorithme converge au vus de l'allure des coefficients de $\alpha$ sur la dernière itération.  

Essayons de trouver l'impact de $\sigma$ et de $nit$ sur la précision du signal reconstruit.

In [ ]:
SIG = [0.001,0.01,0.05,0.1,0.15,1]
NIT = [500, 1000, 5000, 10000, 20000]
#plt.subplots_adjust(hspace=0.8)
for sigma in SIG:
    fig, (AXE) = plt.subplots(1, len(NIT))
    fig.suptitle(r"$\sigma$ = "+ "{:.1e}".format(sigma), fontsize=25)
    fig.set_size_inches(25, 4)
    for i in range(len(NIT)):
        nit = NIT[i]
        axe = AXE[i]
        (alpha,xtilde,CF)=RestoreX(A,y,sigma,nit)
        axe.plot(alpha)
        axe.set_title("nit =" + "{:}".format(nit),y = -0.2,fontsize = 15)
    plt.show()
    print("\n")

On remarque que pour de trop petites valeurs de $\sigma$ comme  1e-3, l'algorithme ne converge pas quelque soit $nit$ (malgré le fait que si $\sigma$ tend vers 0, la solution du problème (4) tend vers une solution du problème (3)).

A partir de $\sigma$  supérieurs à 1e-2 par exemple, on a une convergence qui se fait que si le nombre d'itération $nit$ est assez élevé et $nit$ doit évoluer avec $\sigma$ pour avoir convergence.

**Q9.** Vérifiez que la fonction coût décroit de façon monotone. Quel est le taux de convergence observé ?

Nous reprenons le même test que précedemment mais à la différence, nous allons observer la variation de la fonction coût et ensuite déterminer pour celle-ci le taux de convergence correspondant.

In [ ]:
SIG=[0.001,0.01,0.05,0.1,0.15,1]
NIT=[500, 1000, 5000, 10000, 20000]
from matplotlib.ticker import FixedLocator, MultipleLocator

#plt.subplots_adjust(hspace=0.8)
for sigma in SIG:
    fig, (AXE) = plt.subplots(1, len(NIT))
    fig.suptitle(r"$\sigma$ = "+ "{:.1e}".format(sigma), fontsize=25)
    fig.set_size_inches(25, 4)
    for i in range(len(NIT)):
        nit = NIT[i]
        axe = AXE[i]
        axe.grid()
        (alpha,xtilde,CF)=RestoreX(A,y,sigma,nit)
        axe.plot(CF)
        axe.yaxis.set_major_locator(FixedLocator([min(CF), max(CF)]))
        axe.axhline(y=min(CF),color='r',linewidth=0.5)
        axe.set_title("nit =" + "{:}".format(nit),y=-0.2,fontsize=15)

    plt.show()
    print("\n")

On remarque que  la fonction coût est monotone quelque soit les valeurs de $\sigma$ et de $nit$ .

De plus pour les très petites valeurs de $\sigma$ par exemple 1e-3, la fonction coût n'évolue pas, en effet pour cette valeur de $\sigma$, on avait le vecteur nul pour $\alpha$

Pour des valeurs de $\sigma$ de l'ordre de 1e-2, le courbe est décroissante.

Expérimentalement, on trouve que $\sigma = 0.01$ est le meilleur choix pour avoir une bonne convergence

Ainsi, dans la suite, nous ferons des tests pour $\sigma=0.01$. 


In [ ]:
def Conv_rate(x, x_exact):
    e = [abs(x_ - x_exact) for x_ in x]
    q = [np.log(e[n+1]/e[n])/np.log(e[n]/e[n-1]) for n in range(1, len(e)-1, 1)]
    return np.array(q)

Calculons le taux de convergence qui est définie par :
$$ \lim\limits_{x \rightarrow +\infty} \frac{|x_{k+1}-L|}{|x_{k}-L|} $$

In [ ]:
sigma=0.1
nit=20000
(alpha,xtilde,CF)=RestoreX(A,y,sigma,nit)
L=CF[-1]
Taux_conv=Conv_rate(CF,CF[-1])
plt.plot(Taux_conv)
plt.show()

## Taux de convergence observé


In [ ]:
sigma=0.01
nit=1000
(alpha,xtilde,CF)=RestoreX(A,y,sigma,nit)
x=np.arange(0,len(CF))
coefficients = np.polyfit(x,np.log(CF),1) 
polynomial = np.poly1d(coefficients)
ys = polynomial(x)

plt.figure()
plt.grid()
plt.plot(ys, label="Pente")
plt.semilogy(np.log(CF), label= "Fonction coût (log)")
plt.legend()
plt.show()
print("La courbe semble suivre la fonction",polynomial)
print("Le taux de convergence obtenu est de", "{:.1e}".format(np.abs(coefficients[0]*100)))

In [ ]:
# Taux de convergence 
sigma = 0.01
nit = 1000
(alpha,xtilde,CF)=RestoreX(A,y,sigma,nit)
coefficients = np.polyfit(np.log(CF[0:nit-1]),np.log(CF[1:nit]),1) # Pente de la courbe
polynomial = np.poly1d(coefficients)
ys = polynomial(x)
print(coefficients)
print(polynomial)

plt.figure()
plt.grid()
plt.loglog(np.log(CF[0:nit-1]),np.log(CF[1:nit]), label = "Fonction Coût en log")
plt.legend()
plt.show()
print("Le taux de Convergence obtenu est de", "{:}".format(np.abs(coefficients[0]*100)))

### Interprétation taux de convergence

**Q10.** A partir de combien de mesures pouvez-vous reconstruire exactement le signal $x$ ?

In [ ]:
sigma = 0.01
nit = 5000
for i in [5,10,15,20,25,30,40,50,60,70]:
    p = i*4
    A = np.random.randn(p,n) #La matrice de mesure
    y = A.dot(x0)   #Les mesures
    fig,ax = plt.subplots(1,3,figsize=(15,5))
    fig.suptitle("p ="+"{:}".format(p), fontsize=25)

    (alpha,xtilde,CF) = RestoreX(A,y,sigma,nit)

    k = np.shape(np.where(np.abs(alpha)!=0))[1]
    SHAN = 20*k*np.log(n)
    print("________________________________\n\n")
    print("Constante de Shannon =",SHAN)
    ax[0].plot(t,Psi(alpha))
    ax[0].set_title("Signal reconstruit",y = -0.2)

    ax[1].plot(t,x0,'b')
    ax[1].set_title("Signal initial",y = -0.2)

    ax[2].plot(t,np.abs(x0-Psi(alpha))/np.abs(x0),'y')
    ax[2].set_title("Erreur de reconstruction",y = -0.2)

    print("Erreur = ",npl.norm(x0-Psi(alpha))/npl.norm(x0))
    plt.show()
    print("\n")


### Comparaison avec le théorème de Shannon 

On remarque que lorsque p est trop petit , donc inférieur à 120 on a de très grosses erreurs sur la reconstruction du signal

Et plus p est grand, meilleur sera la reconstruction, cependant on tournera toujours autour de 0.1 en terme d'erreur..

### 4.2. Implémentation de l'itération proximale accelérée

On n'a a aucun moment utilisé la convexité de la fonction $J$ pour définir l'algorithme proximal. Celui-ci est de fait sous-optimal et peut être nettement accéléré. Dans notre cas, l'algorithme accéléré (Nesterov 2007) suit le schéma suivant:

Paramètres en entrée:
* $N$ le nombre d'itérations
* $\alpha^0\in \mathbb{R}^m$ un point initial.

Algorithme
> Poser $B^0=0_\mathbb{R}$, $g^0=0_{\mathbb{R}^m}$, $\alpha=\alpha^0$

> For $k$= $0$ to $N$

>> $t= \frac{2}{L}$ 

>> $a^k = \frac{1}{2}\left(t+\sqrt{t^2+4t B^k}\right)$ 

>> $v^{k} =\mbox{prox}_{B^k\|\cdot\|_1}(\alpha^0-g^k)$ 

>> $w^k = \frac{B^k \alpha^k +a^k v^k}{B^k+a^k}$ 

>>  $\alpha^{k+1} = \mbox{prox}_{\frac{1}{L}\|\cdot\|_1}\left(w^k-\frac{\nabla J(w^k)}{L}\right)$  

>>	$g^{k+1} = g^k + a^k\nabla J(\alpha^{k+1})$ 

>>	$A^{k+1} = B^k+a^k$
 
**Q11.** En vous aidant de ce que vous avez codé dans la partie précédente, implémentez cet algorithme. 

In [ ]:
def Nesterov(A,y,sigma,nit):
    n = np.shape(A)[1]
    x = np.zeros(nit*n) 
    alpha = np.zeros(2*n)
    CF = np.zeros(nit)
    B = 0
    g = np.zeros(2*n)
    iter = 0
    while iter<nit:
        L = 2*sigma*npl.norm(A)**2
        t = 2/L
        a = 1/2*(t+np.sqrt(t**2+4*t*B))
        v = prox(alpha-g,B)
        w = (B*alpha+a*v)/(B+a)
        alpha = prox(w-1/L*PsiT(A.T.dot(A.dot(Psi(w))-y)),1/L)
        x[n*iter:n*(iter+1)] = Psi(alpha)
        g = g+a*PsiT(A.T.dot(A.dot(Psi(alpha))-y))
        B += a
        CF[iter] = npl.norm(alpha,1)+(sigma/2)*npl.norm(A.dot(Psi(alpha))-y)**2
        iter += 1
    return (alpha,x,CF)

**Q12.** Testez le et comparez la rapidité d'execution de l'algorithme précédent et de celui-ci.

In [ ]:
nit = 10000
SIG = [0.05,0.1,0.15,1,10,100]
for sigma in SIG:
    fig,ax = plt.subplots(1,2,figsize=(15,5))
    ax[0].grid()
    ax[1].grid()
    (alpha,xtilde,CF) = RestoreX(A,y,sigma,nit)
    (alpha1,x,CF1) = Nesterov(A,y,sigma,nit)
    ax[0].plot(CF)
    ax[1].plot(CF1)
    ax[0].yaxis.set_major_locator(FixedLocator([min(CF), max(CF)]))
    ax[1].yaxis.set_major_locator(FixedLocator([min(CF1), max(CF1)]))
    ax[0].axhline(y = min(CF),color = 'r',linewidth = 0.5)
    ax[1].axhline(y = min(CF1),color = 'r',linewidth = 0.5)
    ax[0].set_title(r" Algo Proximal avec $\sigma$ = "+ "{:.1e}".format(sigma) + " / nit =" + "{:}".format(nit))

    ax[1].set_title(r"Nesterov avec $\sigma$ = "+ "{:.1e}".format(sigma) + " / nit =" + "{:}".format(nit))

    plt.show()

On remarque que quelque soit la valeur de $\sigma$ l'algorithme de Nesterov  converge plus rapidement et que pour les valeurs de $\sigma$ considérées le minimum de l'algorithme proximal tourne autour des mêmes valeurs (entre $7$ et $8.5$). Cette valeur minimale est donc plus controlée pour Nesterov contrairement à l'algorithme du proximal  pour lequel les valeurs de ce minimum augmentent avec les valeurs de $\sigma$.

Nous allons par les suites tester l'hypothèse de Shannon concernant le nombre de mesures. Pour ce faire, nous fixons $\sigma$ à $0.1$ et $nit$ à $20000$. 

In [ ]:
nit = 20000
sigma = 0.1
for i in [5,10,15,20,25,30,40,50,60,70]:
    p = i*4
    A = np.random.randn(p,n) #La matrice de mesure
    y = A.dot(x0)   #Les mesures
    fig,ax = plt.subplots(1,3,figsize = (15,5))
    fig.suptitle("p = "+"{:}".format(p), fontsize=25)

    (alpha,xtilde,CF) = RestoreX(A,y,sigma,nit)

    k=np.shape(np.where(np.abs(alpha) != 0))[1]
    SHAN=20*k*np.log(n)
    print("________________________________\n\n")
    print("Constante de Shannon = ",SHAN)
    ax[0].plot(t,Psi(alpha))
    ax[0].set_title("Signal reconstruit",y=-0.2)

    ax[1].plot(t,x0,'b')
    ax[1].set_title("Signal initial",y = -0.2)

    ax[2].plot(t,np.abs(x0-Psi(alpha))/np.abs(x0),'y')
    ax[2].set_title("Erreur de reconstruction",y = -0.2)

    print("Erreur = ",npl.norm(x0-Psi(alpha))/npl.norm(x0))
    plt.show()
    print("\n")

Nous pouvons déjà constaté que les résultats sont bien meilleures pour l'algorithme proximal. Pour $p=20$, l'erreur est de 0.84 (d'ordre 1e-1).Cela est visible au niveau du signal obtenu pour cette valeur de $p$. Nous voyons que le signal reconstruit est très différent du signal original. Cette erreur est élevée mais diminue très vite. En effet, nous constatons que nous arrivons à $0.01$ pour $p=20$ et à $0.08$ pour $p=60$. Nous arrivons au final à $0.0015$ pour $p=280$. Ces erreurs sont relativement faibles.Nous pouvons donc dire qu'on arrive assez bien à reconstruire le signal à partir de $p=40$.